Pipeline to pull data from Households Below Average Income data from Stat-Xplore

In [1]:
import os
import json
import pandas as pd
from api import run_statxplore_query

In [2]:
OUTPUT_DIR="../../data/hbai"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
HBAI_JSON = "json/data/HBAI.json"

In [4]:
views = {
    "by_age_category": [
        "str:field:HBAI:V_F_HBAI:TYPE_AGECAT"
    ],
    "by_number_of_children": [
        "str:field:HBAI:V_F_HBAI:NUMBKIDS"
    ],
    "by_age_of_youngest_child": [
        "str:field:HBAI:V_F_HBAI:YOUNGCH"
    ],
    "by_tenure_type": [
        "str:field:HBAI:V_F_HBAI:TENHBAI"
    ],
    "by_savings_and_investments": [
        "str:field:HBAI:V_F_HBAI:CAPITAL"
    ],
    "by_ethnic_group": [
        "str:field:HBAI:V_F_HBAI:ETHGRPHHPUB"
    ],
    "by_marital_status": [
        "str:field:HBAI:V_F_HBAI:COUPLE"
    ]
}

In [5]:
location = "Location in the United Kingdom of the Household of the Individual (please calculate three-year averages - click on i for the correct method)"

def extract_location_code(data):
    # Extract geography code from location
    data["geography_code"] = (
        data[location].str.split(r"[()]", expand=True).iloc[:,1].str.strip()
    )
    data = data.drop(location, axis=1)
    return data

In [6]:
def run_query(dim):
    with open(HBAI_JSON) as json_file:
        query = json.load(json_file)
    query["dimensions"] += [dim]
    return run_statxplore_query(query)


def reshape_statxplore_result(data):
    # Convert wide to long format
    data = data.melt(ignore_index=False, var_name='income_status')
    # Add in a variable name
    data['variable_name'] = 'population'
    # Split geo codes and remove bracketed statements from colum names
    data.index = data.index.set_levels(
            data.index.levels[1].str.strip(')').str.split('(').map(lambda l: l[1]).values, level=1
        ).set_names(
            'geography_code', level=1
        ).set_names(
            data.index.names[2].split('(')[0].strip(),
            level=2
        )
    return data

def three_year_average(data, date_name='Financial Year', round=False):
    '''
    Three year rolling average
    '''
    # Dynamically get a list of variables to group by - everything apart from date and value.
    data.reset_index(inplace=True)
    colnames = data.columns.to_list()
    if "Financial Year" not in colnames:
        return print("Check the date name of the input dataframe - should be financial year", colnames)
    else: 
        colnames.remove('Financial Year')
    if "value" in colnames:
        colnames.remove('value')
        groupby = colnames
    else: 
        return print('no value column in the data')
    #removing missing data
    data = data[data[date_name] != "2020-21"]
    
    #getting a list of unique dates
    dates = data[date_name].unique()
    frames = []
    for i in range(len(dates)-1):
        #accounting for missing data in accordance with HBAI data guidance.
        if dates[0] == '2018-19':
            most_recent_dates = ['2018-19', '2019-20']
        elif dates[0] == '2019-20':
            most_recent_dates = ['2019-20', '2021-22']
        elif i != len(dates)-1:
            most_recent_dates = list(dates[i:(i+3)])

        new_data = data.loc[data[date_name].isin(most_recent_dates)].copy()
        new_data = new_data.groupby(groupby).sum(numeric_only=True) / len(most_recent_dates)
        new_data["date"] = f'{most_recent_dates[0]} - {most_recent_dates[len(most_recent_dates)-1]}'
        #new_data["variable_name"] = variable_name
        # rounding to nearest 0.1mil, according to user guidance.
        if round:
            new_data = new_data.round(-5)
        frames.append(new_data.reset_index())
        #i+=1
        #print(i)
    #concatenating the frames
    return pd.concat(frames, axis=0)

def get_percentages(data):
    data['population'] = data.groupby(['date', 'geography_code'])['value'].transform('sum')
    data['percent'] = (data['value'] / data['population'] * 100).round(0)
    data.set_index('geography_code', inplace=True)
    return data

for name, dim in views.items():
    data = run_query(dim).pipe(reshape_statxplore_result).pipe(three_year_average).pipe(get_percentages)
    data.to_csv(f"{OUTPUT_DIR}/{name}.csv")

In [7]:
print(data.to_csv())

geography_code,Marital Status of Adults and Type of Couple in the Family of the Individual,income_status,variable_name,value,date,population,percent
E12000001,Couple,In low income (below threshold),population,608579.6666666666,1994-95 - 1996-97,2543594.0,24.0
E12000001,Couple,Not in low income (at or above threshold),population,1935014.3333333333,1994-95 - 1996-97,2543594.0,76.0
E12000001,Single,In low income (below threshold),population,0.0,1994-95 - 1996-97,2543594.0,0.0
E12000001,Single,Not in low income (at or above threshold),population,0.0,1994-95 - 1996-97,2543594.0,0.0
E12000002,Couple,In low income (below threshold),population,1389112.0,1994-95 - 1996-97,6719032.333333333,21.0
E12000002,Couple,Not in low income (at or above threshold),population,5329920.333333333,1994-95 - 1996-97,6719032.333333333,79.0
E12000002,Single,In low income (below threshold),population,0.0,1994-95 - 1996-97,6719032.333333333,0.0
E12000002,Single,Not in low income (at or above threshold),population,0.